# Homework Assignment #03

## Global Alignment with Affine Gap Penalties

In [1]:
import random
import warnings
import numpy as np

Let's implement basic global alignment algorithm with random path reconstruction between all possible directions

In [2]:
def align_global(s1, s2, penalty_matrix, alphabet='ACGT-'):
    """
    s1 - string alongside rows of the alignment matrix
    s2 - string alongside columns of the alignment matrix
    pentalty_matrix - numpy matrix with penalty wegiths, 
        where raws and columns correspond to the alphabet
    alphabet - string of possible symbols
    """
    m = np.zeros((len(s1)+1, len(s2)+1))
    
    def get_penalty(c1, c2):
        return penalty_matrix[alphabet.index(c1), alphabet.index(c2)]
    
    # First raw and column filled with beta penalties
    for i in range(1, len(s1)+1):
        m[i, 0] = m[i-1, 0] - get_penalty(s1[i-1], '-')
    for j in range(1, len(s2)+1):
        m[0, j] = m[0, j-1] - get_penalty('-', s2[j-1])
        
    # Fill matrix raw by raw
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            m[i, j] = max(
                m[i-1, j-1] - get_penalty(s1[i-1], s2[j-1]),
                m[i, j-1] - get_penalty('-', s2[j-1]),
                m[i-1, j] - get_penalty(s1[i-1], '-')
            )
                        
    # Reconstruct alignment
    s1_aligned = ''
    s2_aligned = ''
    path = [(len(s1), len(s2))]
    i, j = len(s1), len(s2)
    while i > 0 or j > 0:
        score = m[i, j]
        # Choose randomly between all possible directions
        directions = [score == m[i-1, j-1] - get_penalty(s1[i-1], s2[j-1]),
                      score == m[i-1, j] - get_penalty(s1[i-1], '-'),
                      score == m[i, j-1] - get_penalty('-', s2[j-1])]
        opts = [i for i, cond in enumerate(directions) if cond]
        opt = random.choice(opts)
        # Make step towards chosen direction
        if opt == 1:
            s2_aligned = '-' + s2_aligned
            s1_aligned = s1[i-1] + s1_aligned
            i -= 1
        elif opt == 0:
            s1_aligned = s1[i-1] + s1_aligned
            s2_aligned = s2[j-1] + s2_aligned
            i -= 1
            j -= 1
        else:
            s1_aligned = '-' + s1_aligned
            s2_aligned = s2[j-1] + s2_aligned
            j -= 1
        path.insert(0, (i, j))
                
    return m, s1_aligned, s2_aligned, path

Global alignment with affine gaps implementation

In [7]:
def align_affine_gaps(s1, s2, penalty_matrix, sigma, ro, alphabet='ACGT-'):
    """
    s1 - string alongside rows of the alignment matrix
    s2 - string alongside columns of the alignment matrix
    pentalty_matrix - numpy matrix with penalty wegiths, 
        where raws and columns correspond to the alphabet
    sigma - penalty for gap continuation (positive)
    ro - penalty for opening gap (positive)
    alphabet - string of possible symbols
    """
    
    # Three matrices with scores. 
    # Upper row of m_down matrix and leftmost column of m_right matrix shold have hight negative score for 
    # exception of visiting these cells
    m_diag = np.ones((len(s1)+1, len(s2)+1)) * -np.inf
    m_right = np.ones((len(s1)+1, len(s2)+1)) * -np.inf
    m_down = np.ones((len(s1)+1, len(s2)+1)) * -np.inf
    
    # Three matrices for path reconstuction. 1 would indicate movement inside the same matrix
    m_rec_diag = np.ones((len(s1)+1, len(s2)+1))
    m_rec_right = np.ones((len(s1)+1, len(s2)+1))
    m_rec_down = np.ones((len(s1)+1, len(s2)+1))
    
    def get_penalty(c1, c2):
        return penalty_matrix[alphabet.index(c1), alphabet.index(c2)]
    
    # Fill first raw of the down matrix with penalties
    m_down[0, 0] = 0
    m_diag[0, 0] = 0
    m_down[1, 0] = -ro
    m_diag[1, 0] = -ro
    for i in range(2, len(s1)+1):
        m_down[i, 0] = m_down[i-1, 0] - sigma
        m_diag[i, 0] = m_diag[i-1, 0] - sigma
    # Fill first column of the right matrix with penalties
    m_right[0, 0] = 0
    m_right[0, 1] = -ro
    m_diag[0, 1] = -ro
    for j in range(2, len(s2)+1):
        m_right[0, j] = m_right[0, j-1] - sigma
        m_diag[0, j] = m_diag[0, j-1] - sigma
        
    # Fill matrix raw by raw
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            # Down matrix
            m_down[i, j] = max(
                m_down[i-1, j] - sigma,
                m_diag[i-1, j] - (sigma + ro)
            )
            # Right matrix
            m_right[i, j] = max(
                m_right[i, j-1] - sigma,
                m_diag[i, j-1] - (sigma + ro)
            )
            # Diag matrix
            m_diag[i, j] = max(
                m_diag[i-1, j-1] - get_penalty(s1[i-1], s2[j-1]),
                m_down[i, j],
                m_right[i, j]
            )
            
            # Track path
            if m_down[i, j] == m_down[i-1, j] - sigma:
                m_rec_down[i, j] = 1
            else:
                m_rec_down[i, j] = 2
            if m_right[i, j] == m_right[i, j-1] - sigma:
                m_rec_right[i, j] = 1
            else:
                m_rec_right[i, j] = 2
            if m_diag[i, j] == m_diag[i-1, j-1] - get_penalty(s1[i-1], s2[j-1]):
                m_rec_diag[i, j] = 1
            elif m_diag[i, j] == m_down[i, j]:
                m_rec_diag[i, j] = 2
            else:
                m_rec_diag[i, j] = 3
            
    # Reconstruct alignment
    s1_aligned = ''
    s2_aligned = ''
    i, j = len(s1), len(s2)
    
    cur_matrix = 'diag'   # Track current matrix 
    prev_matrix = None    # Track previous visited matrix to catch direct jumps between m_right and m_down matrices
    while i > 0 and j > 0:
        if cur_matrix == 'diag':
            if m_rec_diag[i, j] == 1:
                s1_aligned = s1[i-1] + s1_aligned
                s2_aligned = s2[j-1] + s2_aligned
                # Move withing matrix only if we perform no jumps
                i -= 1; j -= 1
            else:
                if prev_matrix != 'diag':
                    msg = "New gap was opened while no steps in middle matrix were performed. Weights are initialized incorrect."
                    warnings.warn(msg)
                if m_rec_diag[i, j] == 2:
                    cur_matrix = 'down'
                elif m_rec_diag[i, j] == 3:
                    cur_matrix = 'right'
            prev_matrix = 'diag'
        elif cur_matrix == 'down':
            prev_matrix = 'down'
            s2_aligned = '-' + s2_aligned
            s1_aligned = s1[i-1] + s1_aligned
            if m_rec_down[i, j] == 2:
                cur_matrix = 'diag'
            i -= 1
        else:
            prev_matrix = 'right'
            s1_aligned = '-' + s1_aligned
            s2_aligned = s2[j-1] + s2_aligned    
            if m_rec_right[i, j] == 2:
                cur_matrix = 'diag'
            j -= 1
                
    return s1_aligned, s2_aligned, path

### Preliminary test for correctness
Let's consider basic tests where global alignment can insert multiple gaps while alignment with affine penalties would prefer only one continous gap

In [8]:
# ACGT + gap
penalty_weights = np.array([
    [-1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, -1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, -1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, -1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.0]
])

print("Global Alignment")
matrix, s1_align, s2_align, path = align_global('AACCTTTTTAG', 'AACCTTAG', penalty_weights)
print("S1: ", s1_align)
print("S2: ", s2_align)
print("\nAffine Gaps Alignment")
s1_align, s2_align, path = align_affine_gaps('AACCTTTTTAG', 'AACCTTAG', penalty_matrix=penalty_weights, sigma=1, ro=2)
print("S1: ", s1_align)
print("S2: ", s2_align)

Global Alignment
S1:  AACCTTTTTAG
S2:  AACCT---TAG

Affine Gaps Alignment
S1:  AACCTTTTTAG
S2:  AACC---TTAG


In [6]:
s1 = 'GATCTCCAG'
s2 = 'GACAG'
print("Global Alignment")
matrix, s1_align, s2_align, path = align_global(s1, s2, penalty_weights)
print("S1: ", s1_align)
print("S2: ", s2_align)
print("\nAffine Gaps Alignment")
s1_align, s2_align, path = align_affine_gaps(s1, s2, penalty_weights, sigma=1, ro=2)
print("S1: ", s1_align)
print("S2: ", s2_align)

Global Alignment
S1:  GATCTCCAG
S2:  GA-C---AG

Affine Gaps Alignment
S1:  -CCAG
S2:  GACAG


### Test 1.

* Match bonus: 1
* Mimatch penalty: -1
* Opening gap penalty: 0
* Gap continuation penalty: -1

In [6]:
s1 = 'TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC'
s2 = 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC'

ro = 0
sigma = 1
s1_align, s2_align, path = align_affine_gaps(s1, s2, penalty_matrix=penalty_weights, sigma=sigma, ro=ro)
print("S1: ", s1_align)
print("S2: ", s2_align)

S1:  T--CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
S2:  TTGCCGCCGTCGT-TTTCAG----CA-G-TTATGTCAGA-TC


Inference: many matches exists, as well as multiple gaps as opening gap not punished

### Test 2

* Match bonus: 1
* Mistmatch penalty: -1
* Opening gap penalty: -100
* Gap continuation penalty: 0.01

In [7]:
ro = 100
sigma = 0.01
s1_align, s2_align, path = align_affine_gaps(s1, s2, penalty_matrix=penalty_weights, sigma=sigma, ro=ro)
print("S1: ", s1_align)
print("S2: ", s2_align)

S1:  TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC
S2:  AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC


Inference: no gaps were created as creating gap was punished too hard

### Test 3

* Match bonus: 1
* Mistmatch penalty: -1
* Opening gap penalty: +0.5
* Gap continuation penalty: -0.3

In [8]:
ro = -0.5
sigma = 0.3
s1_align, s2_align, path = align_affine_gaps(s1, s2, penalty_matrix=penalty_weights, sigma=sigma, ro=ro)
print("S1: ", s1_align)
print("S2: ", s2_align)

S1:  T---C--C--CAGTTATGTCAGGGGACACG--A-GCATGCAGA-GAC
S2:  TTGCCGCCGTC-GTT-T-TCA---G-CA-GTTATG--T-CAGAT--C


/Users/igor/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:96: UserWarning: New gap was opened while no steps in middle matrix were performed. Weights are initialized incorrect.


Inference: direct jumps between upper and down matrices were witnessed, too many gaps were created